In [2]:
import json
import re

with open('67_2014_QH13_259729_only_dieu_jurisdict.json', 'r', encoding="utf-8") as json_file:
	data = json.load(json_file)
	main_data = data["jurisdict"]
with open('67_2014_QH13_259729_only_dieu_index_another_way.json', 'r', encoding="utf-8") as json_file_2:
	ref_data = json.load(json_file_2)

allow_appear_before_keyword = ["được quyết định bởi"]
not_allow_appear_before_keyword = ["không được quyết định bởi"]
allow_appear_after_keyword = ["quy định", "quyết định", "có thẩm quyền", "quy định chi tiết"]
not_allow_appear_after_keyword = ["không được ban hành"]

process_data = []
agency_and_jurisdiction = {
	"allow" : {},
	"not_allow" : {}
}

before_keyword_list = allow_appear_before_keyword + not_allow_appear_before_keyword
after_keyword_list = allow_appear_after_keyword + not_allow_appear_after_keyword

def extract_numbers(text, start, end):
    # Slicing the text from start to end index
    sliced_text = text[start:end+1]
    # Using regular expression to find all numbers in the sliced text
    numbers = re.findall(r'\b\d+\b', sliced_text)
    # Converting found numbers from strings to integers
    return [number for number in numbers]

def extract_characters(text, start_index, end_index):
    # Extract the substring based on the provided indices
    substring = text[start_index:end_index + 1]

    # Use regular expression to find all standalone characters
    # \b is a word boundary, \w is any word character, and {1} specifies exactly one occurrence of a word character
    import re
    pattern = r'\b\w{1}\b'
    alone_characters = re.findall(pattern, substring)

    return alone_characters

def extract_reference(index, text):
	dieu_list = []
	khoan_list = []
	diem_list = []
	index_list = index.split(".")
	text_lower = text.lower()
	if "điều này" in text_lower:
        # Get first appear of "Điều này"
		dieu_nay_index = text_lower.find("điều này")
		if "khoản" in text_lower and text_lower.find("khoản") < dieu_nay_index:
			khoan_index = text_lower.find("khoản")
			if "điểm" in text_lower and text_lower.find("điểm") < khoan_index:
				# Get first appear Điểm index
				diem_index = text_lower.find("điểm")
				diem_list = extract_characters(text_lower, diem_index, khoan_index)
			khoan_list = extract_numbers(text_lower, khoan_index, dieu_nay_index)
			dieu_list.append(index_list[0])
		else:
			dieu_list.append(index_list[0])
	else:
		if "điều" in text_lower:
            # Get first appear of "Điều"
			dieu_index = text_lower.find("điều")
			if "khoản" in text_lower and text_lower.find("khoản") < dieu_index:
                # Get first appear Khoản index
				khoan_index = text_lower.find("khoản")
				if "điểm" in text_lower and text_lower.find("điểm") < khoan_index:
					# Get first appear Điểm index
					diem_index = text_lower.find("điểm")
					diem_list = extract_characters(text_lower, diem_index, khoan_index)
                # for character in text from khoan_index to dieu_index
				khoan_list = extract_numbers(text_lower, khoan_index, dieu_index)
				dieu_list = extract_numbers(text_lower, dieu_index, dieu_index+10)
			else:
				dieu_list = extract_numbers(text_lower, dieu_index, len(text_lower)-1)
				if len(index_list) > 10:
					dieu_list = dieu_list[:10]
		else:
			if "khoản" in text_lower:
                # Get first appear Khoản index
				khoan_index = text_lower.find("khoản")
				if "điểm" in text_lower and text_lower.find("điểm") < khoan_index:
					# Get first appear Điểm index
					diem_index = text_lower.find("điểm")
					diem_list = extract_characters(text_lower, diem_index, khoan_index)
                # for character in text from khoan_index to end of text
				khoan_list = extract_numbers(text_lower, khoan_index, len(text_lower)-1)
				if len(index_list) > 10:
					khoan_list = khoan_list[:10]
				dieu_list.append(index_list[0])
			else:
				if "điểm" in text_lower:
					# Get first appear Điểm index
					diem_index = text_lower.find("điểm")
					diem_list = extract_numbers(text_lower, diem_index, len(text_lower)-1)
					if len(index_list) > 10:
						diem_list = diem_list[:10]
					khoan_list.append(index_list[1])
					dieu_list.append(index_list[0])
	return dieu_list, khoan_list, diem_list

def get_reference(index, text):
	dieu_list, khoan_list, diem_list = extract_reference(index, text)
	if len(dieu_list) == 1:
		if len(khoan_list) > 1:
			for khoan in khoan_list:
				ref_index = f"{dieu_list[0]}.{khoan}.0"
				return ref_data[ref_index] if ref_index in ref_data else ""
		else:
			for diem in diem_list:
				ref_index = f"{dieu_list[0]}.{khoan_list[0]}.{diem}"
				return ref_data[ref_index] if ref_index in ref_data else ""
	else:
		for dieu in dieu_list:
			ref_index = f"{dieu}.0.0"
			return ref_data[ref_index] if ref_index in ref_data else ""
		
def get_higher_level_index(index):
	index_list = index.split(".")
	if index_list[2] != "0":
		return ref_data[f"{index_list[0]}.{index_list[1]}.0"]
	elif index_list[2] == "0" and index_list[1] != "0":
		return ref_data[f"{index_list[0]}.0.0"]
	
for element in main_data:
	for before_keyword in before_keyword_list:
		# if before_keyword in element["text"] and before_keyword appear before element["agency"], extract the text before before_keyword from element["text"]
		if before_keyword in element["text"] and element["text"].find(before_keyword) < element["text"].find(element["agency"]):
			index = element["text"].find(before_keyword)
			process_data.append({
				"index": element["index"], 
				"text": element["text"],
				"agency": element["agency"],
				"jurisdiction": element["text"][:index],
				"authority": before_keyword,
				"reference": get_reference(element["index"], element["text"][:index]),
				"higher_level_reference": get_higher_level_index(element["index"])
				})
			jurisdiction_text = element["text"][:index]
			reference_text = get_reference(element["index"], element["text"][:index])
			higher_level_reference_text = get_higher_level_index(element["index"])
			get_reference(element["index"], element["text"][:index])
			# if element["agency"] is an existed key in agency_and_jurisdiction, append the jurisdiction to the existed key
			if before_keyword in allow_appear_before_keyword:
				if element["agency"] in agency_and_jurisdiction["allow"]:
					agency_and_jurisdiction["allow"][element["agency"]].append(
						jurisdiction_text + " " + 
						(reference_text if reference_text is not None else "") + " " + 
						(higher_level_reference_text if higher_level_reference_text is not None else "")
					)
				else:
					agency_and_jurisdiction["allow"][element["agency"]] = [element["text"][:index]]
			else:
				if element["agency"] in agency_and_jurisdiction["not_allow"]:
					agency_and_jurisdiction["not_allow"][element["agency"]].append(
						jurisdiction_text + " " + 
						(reference_text if reference_text is not None else "") + " " + 
						(higher_level_reference_text if higher_level_reference_text is not None else "")
					)
				else:
					agency_and_jurisdiction["not_allow"][element["agency"]] = [
						jurisdiction_text + " " +
						(reference_text if reference_text else "") + " " +
						(higher_level_reference_text if higher_level_reference_text else "")
					]

	for after_keyword in after_keyword_list:
		# if after_keyword in element["text"] and after_keyword appear after element["agency"], extract the text after after_keyword from element["text"]
		if after_keyword in element["text"] and element["text"].find(after_keyword) > element["text"].find(element["agency"]):
			index = element["text"].find(after_keyword)
			process_data.append({
				"index": element["index"], 
				"text": element["text"],
				"agency": element["agency"],
				"jurisdiction": element["text"][index + len(after_keyword):],
				"authority": after_keyword,
				"reference": get_reference(element["index"], element["text"][index + len(after_keyword):]),
				"higher_level_reference": get_higher_level_index(element["index"])
			})
			jurisdiction_text = element["text"][index + len(after_keyword):]
			reference_text = get_reference(element["index"], element["text"][index + len(after_keyword):])
			higher_level_reference_text = get_higher_level_index(element["index"])
			# get_reference(element["index"], element["text"][index + len(after_keyword):])
			if after_keyword in allow_appear_after_keyword:
				if element["agency"] in agency_and_jurisdiction["allow"]:
					agency_and_jurisdiction["allow"][element["agency"]].append(
						jurisdiction_text + " " +
						(reference_text if reference_text is not None else "") + " " +
						(higher_level_reference_text if higher_level_reference_text is not None else "")
					)
				else:
					agency_and_jurisdiction["allow"][element["agency"]] = [
						jurisdiction_text + " " +
						(reference_text if reference_text else "") + " " +
						(higher_level_reference_text if higher_level_reference_text else "") + " "
					]
			else:
				if element["agency"] in agency_and_jurisdiction["not_allow"]:
					agency_and_jurisdiction["not_allow"][element["agency"]].append(
						(jurisdiction_text if jurisdiction_text is not None else "") + " " +
						(reference_text if reference_text is not None else "") + " " +
						(higher_level_reference_text if higher_level_reference_text is not None else "")
					)
				else:
					agency_and_jurisdiction["not_allow"][element["agency"]] = [
						jurisdiction_text + " " +
						(reference_text if reference_text else "") + " " +
						(higher_level_reference_text if higher_level_reference_text else "")
					]

# save the data to json file
with open('67_2014_QH13_259729_only_dieu_jurisdict_process.json', 'w', encoding="utf-8") as json_file_3:
	json.dump({"jurisdiction" : process_data}, json_file_3, ensure_ascii=False, indent=4)

with open('67_2014_QH13_259729_only_dieu_jurisdict_agency_jurisdiction.json', 'w', encoding="utf-8") as json_file_4:
	json.dump(agency_and_jurisdiction, json_file_4, ensure_ascii=False, indent=4)

In [14]:
state_agencies_vn = [
    "quốc hội",
    "ủy ban thường vụ quốc hội",
    "đoàn chủ tịch ủy ban trung ương mặt trận tổ quốc việt nam",
    "chính phủ",
    "chủ tịch nước",
    "thủ tướng chính phủ",
    "hội đồng thẩm phán tòa án nhân dân tối cao",
    "tòa án nhân dân tối cao",
    "viện kiểm sát nhân dân tối cao",
    "tổng kiểm toán nhà nước",
    "bộ, cơ quan ngang bộ",
    "bộ quốc phòng",
    "bộ công an",
    "bộ ngoại giao",
    "bộ tư pháp",
    "bộ tài chính",
    "bộ công thương",
    "bộ lao động thương binh và xã hội",
    "bộ giao thông vận tải",
    "bộ xây dựng",
    "bộ thông tin và truyền thông",
    "bộ giáo dục và đào tạo",
    "bộ nông nghiệp và phát triển nông thôn",
    "bộ kế hoạch và đầu tư",
    "bộ nội vụ",
    "bộ y tế",
    "bộ khoa học và công nghệ",
    "bộ văn hóa thể thao và du lịch",
    "bộ tài nguyên và môi trường",
    "văn phòng chính phủ",
    "thanh tra chính phủ",
    "ngân hàng nhà nước",
    "ủy ban dân tộc",
    "hội đồng nhân dân",
    "ủy ban nhân dân",
]

In [15]:
from owlready2 import *

onto = get_ontology("http://test.org/onto.owl")

with onto:
	class Agency(Thing):
		pass

with onto:
	class Jurisdiction(Thing):
		pass

for agency_name in state_agencies_vn:
    agency_instance = Agency(name=agency_name)

# create object property hasJurisdiction with domain is Agency and range is Jurisdiction
with onto:
	class hasJurisdiction(ObjectProperty):
		domain = [Agency]
		range = [Jurisdiction]

with onto:
	class hasNoJurisdiction(ObjectProperty):
		domain = [Agency]
		range = [Jurisdiction]

# create an instance of class Jurisdiction with the name is "thuế thu nhập cá nhân" 
jurisdiction_instance = Jurisdiction(name="thuế thu nhập cá nhân")
# class Agency has an instance name "quốc hội", assert object property hasNoJurisdiction with the range is "thuế thu nhập cá nhân"
# get the object named "quốc_hội" from the ontology
agency_instance = onto.search_one(iri="*quốc hội")
print(agency_instance)
agency_instance.hasNoJurisdiction = [jurisdiction_instance]

temp = agency_instance.hasNoJurisdiction
for i in temp:
	print(i.name)

onto.save(file="my_ontology.owl", format="rdfxml")

test.quốc hội
thuế thu nhập cá nhân


In [19]:
from owlready2 import *

onto = get_ontology("http://test.org/onto.owl")

with onto:
	class Agency(Thing):
		pass

with onto:
	class Jurisdiction(Thing):
		pass

for agency_name in state_agencies_vn:
    agency_instance = Agency(name=agency_name)

# create object property hasJurisdiction with domain is Agency and range is Jurisdiction
with onto:
	class hasJurisdiction(ObjectProperty):
		domain = [Agency]
		range = [Jurisdiction]

with onto:
	class hasNoJurisdiction(ObjectProperty):
		domain = [Agency]
		range = [Jurisdiction]

onto.save(file="authority_ontology.owl", format="rdfxml")

In [18]:
from owlready2 import *

onto = get_ontology("my_ontology.owl")

onto.load()

print(list(onto.classes()))
test = onto.get_namespace("http://test.org/onto.owl#")
# create a new instance of class Jurisdiction with the name is "giá cước"
jurisdiction_instance = test.Jurisdiction(name="giá cước")

agency = onto.search_one(iri="*bộ công an")

agency.hasNoJurisdiction = [jurisdiction_instance]

temp = agency.hasNoJurisdiction

for i in temp:
	print(i.name)

[test.Agency, test.Jurisdiction]
giá cước


In [ ]:
from owlready2 import *

onto = get_ontology("authority_ontology.owl")

onto.load()

# print all subclasses of Thing
# print(list(onto.Thing.subclasses()))
for onto_class in onto.classes():
    print(onto_class.name)

In [1]:
import json
import re
from owlready2 import *

with open('67_2014_QH13_259729_only_dieu_jurisdict.json', 'r', encoding="utf-8") as json_file:
	data = json.load(json_file)
	main_data = data["jurisdict"]
with open('67_2014_QH13_259729_only_dieu_index_another_way.json', 'r', encoding="utf-8") as json_file_2:
	ref_data = json.load(json_file_2)

allow_appear_before_keyword = ["được quyết định bởi"]
not_allow_appear_before_keyword = ["không được quyết định bởi"]
allow_appear_after_keyword = ["quyết định", "có thẩm quyền", "quy định chi tiết"]
not_allow_appear_after_keyword = ["không được ban hành"]

process_data = []
agency_and_jurisdiction = {
	"allow" : {},
	"not_allow" : {}
}

before_keyword_list = allow_appear_before_keyword + not_allow_appear_before_keyword
after_keyword_list = allow_appear_after_keyword + not_allow_appear_after_keyword

# read authority_ontology
onto = get_ontology("authority_ontology.owl")

onto.load()

test = onto.get_namespace("http://test.org/onto.owl#")

def extract_numbers(text, start, end):
    # Slicing the text from start to end index
    sliced_text = text[start:end+1]
    # Using regular expression to find all numbers in the sliced text
    numbers = re.findall(r'\b\d+\b', sliced_text)
    # Converting found numbers from strings to integers
    return [number for number in numbers]

def extract_characters(text, start_index, end_index):
    # Extract the substring based on the provided indices
    substring = text[start_index:end_index + 1]

    # Use regular expression to find all standalone characters
    # \b is a word boundary, \w is any word character, and {1} specifies exactly one occurrence of a word character
    import re
    pattern = r'\b\w{1}\b'
    alone_characters = re.findall(pattern, substring)

    return alone_characters

def extract_reference(index, text):
	dieu_list = []
	khoan_list = []
	diem_list = []
	index_list = index.split(".")
	text_lower = text.lower()
	if "điều này" in text_lower:
        # Get first appear of "Điều này"
		dieu_nay_index = text_lower.find("điều này")
		if "khoản" in text_lower and text_lower.find("khoản") < dieu_nay_index:
			khoan_index = text_lower.find("khoản")
			if "điểm" in text_lower and text_lower.find("điểm") < khoan_index:
				# Get first appear Điểm index
				diem_index = text_lower.find("điểm")
				diem_list = extract_characters(text_lower, diem_index, khoan_index)
			khoan_list = extract_numbers(text_lower, khoan_index, dieu_nay_index)
			dieu_list.append(index_list[0])
		else:
			dieu_list.append(index_list[0])
	else:
		if "điều" in text_lower:
            # Get first appear of "Điều"
			dieu_index = text_lower.find("điều")
			if "khoản" in text_lower and text_lower.find("khoản") < dieu_index:
                # Get first appear Khoản index
				khoan_index = text_lower.find("khoản")
				if "điểm" in text_lower and text_lower.find("điểm") < khoan_index:
					# Get first appear Điểm index
					diem_index = text_lower.find("điểm")
					diem_list = extract_characters(text_lower, diem_index, khoan_index)
                # for character in text from khoan_index to dieu_index
				khoan_list = extract_numbers(text_lower, khoan_index, dieu_index)
				dieu_list = extract_numbers(text_lower, dieu_index, dieu_index+10)
			else:
				dieu_list = extract_numbers(text_lower, dieu_index, len(text_lower)-1)
				if len(index_list) > 10:
					dieu_list = dieu_list[:10]
		else:
			if "khoản" in text_lower:
                # Get first appear Khoản index
				khoan_index = text_lower.find("khoản")
				if "điểm" in text_lower and text_lower.find("điểm") < khoan_index:
					# Get first appear Điểm index
					diem_index = text_lower.find("điểm")
					diem_list = extract_characters(text_lower, diem_index, khoan_index)
                # for character in text from khoan_index to end of text
				khoan_list = extract_numbers(text_lower, khoan_index, len(text_lower)-1)
				if len(index_list) > 10:
					khoan_list = khoan_list[:10]
				dieu_list.append(index_list[0])
			else:
				if "điểm" in text_lower:
					# Get first appear Điểm index
					diem_index = text_lower.find("điểm")
					diem_list = extract_numbers(text_lower, diem_index, len(text_lower)-1)
					if len(index_list) > 10:
						diem_list = diem_list[:10]
					khoan_list.append(index_list[1])
					dieu_list.append(index_list[0])
	return dieu_list, khoan_list, diem_list

def get_reference(index, text):
	dieu_list, khoan_list, diem_list = extract_reference(index, text)
	if len(dieu_list) == 1:
		if len(khoan_list) > 1:
			for khoan in khoan_list:
				ref_index = f"{dieu_list[0]}.{khoan}.0"
				return ref_data[ref_index]
		else:
			for diem in diem_list:
				ref_index = f"{dieu_list[0]}.{khoan_list[0]}.{diem}"
				return ref_data[ref_index]
	else:
		for dieu in dieu_list:
			ref_index = f"{dieu}.0.0"
			return ref_data[ref_index]
		
def get_higher_level_index(index):
	index_list = index.split(".")
	if index_list[2] != "0":
		return ref_data[f"{index_list[0]}.{index_list[1]}.0"]
	elif index_list[2] == "0" and index_list[1] != "0":
		return ref_data[f"{index_list[0]}.0.0"]
	
for element in main_data:
	for before_keyword in before_keyword_list:
		if before_keyword in element["text"] and element["text"].find(before_keyword) < element["text"].find(element["agency"]):
			index = element["text"].find(before_keyword)
			jurisdiction_text = element["text"][:index]
			reference_text = get_reference(element["index"], element["text"][:index])
			higher_level_reference_text = get_higher_level_index(element["index"])
			get_reference(element["index"], element["text"][:index])
			if before_keyword in allow_appear_before_keyword:
				jurisdiction_instance = test.Jurisdiction(
					name=jurisdiction_text + " " + 
						(reference_text if reference_text is not None else "") + " " + 
						(higher_level_reference_text if higher_level_reference_text is not None else "")
					)
				agency_instance = onto.search_one(iri=f"*{element['agency']}")
				agency_instance.hasJurisdiction.append(jurisdiction_instance)
				# create an individual of the class with the name of the agency, with the pro
				if element["agency"] in agency_and_jurisdiction["allow"]:
					agency_and_jurisdiction["allow"][element["agency"]].append(
						jurisdiction_text + " " + 
						(reference_text if reference_text is not None else "") + " " + 
						(higher_level_reference_text if higher_level_reference_text is not None else "")
					)
				else:
					agency_and_jurisdiction["allow"][element["agency"]] = [element["text"][:index]]
			else:
				jurisdiction_instance = test.Jurisdiction(
					name=jurisdiction_text + " " + 
						(reference_text if reference_text is not None else "") + " " + 
						(higher_level_reference_text if higher_level_reference_text is not None else "")
					)
				agency_instance = onto.search_one(iri=f"*{element['agency']}")
				agency_instance.hasNoJurisdiction.append(jurisdiction_instance)
				if element["agency"] in agency_and_jurisdiction["not_allow"]:
					agency_and_jurisdiction["not_allow"][element["agency"]].append(
						jurisdiction_text + " " + 
						(reference_text if reference_text is not None else "") + " " + 
						(higher_level_reference_text if higher_level_reference_text is not None else "")
					)
				else:
					agency_and_jurisdiction["not_allow"][element["agency"]] = [
						jurisdiction_text + " " +
						(reference_text if reference_text else "") + " " +
						(higher_level_reference_text if higher_level_reference_text else "")
					]

	for after_keyword in after_keyword_list:
		if after_keyword in element["text"] and element["text"].find(after_keyword) > element["text"].find(element["agency"]):
			index = element["text"].find(after_keyword)
			jurisdiction_text = element["text"][index + len(after_keyword):]
			reference_text = get_reference(element["index"], element["text"][index + len(after_keyword):])
			higher_level_reference_text = get_higher_level_index(element["index"])
			# get_reference(element["index"], element["text"][index + len(after_keyword):])
			if after_keyword in allow_appear_after_keyword:
				jurisdiction_instance = test.Jurisdiction(
					name=jurisdiction_text + " " + 
						(reference_text if reference_text is not None else "") + " " + 
						(higher_level_reference_text if higher_level_reference_text is not None else "")
					)
				agency_instance = onto.search_one(iri=f"*{element['agency']}")
				agency_instance.hasJurisdiction.append(jurisdiction_instance)
				if element["agency"] in agency_and_jurisdiction["allow"]:
					agency_and_jurisdiction["allow"][element["agency"]].append(
						jurisdiction_text + " " +
						(reference_text if reference_text is not None else "") + " " +
						(higher_level_reference_text if higher_level_reference_text is not None else "")
					)
				else:
					agency_and_jurisdiction["allow"][element["agency"]] = [
						jurisdiction_text + " " +
						(reference_text if reference_text else "") + " " +
						(higher_level_reference_text if higher_level_reference_text else "") + " "
					]
			else:
				jurisdiction_instance = test.Jurisdiction(
					name=jurisdiction_text + " " + 
						(reference_text if reference_text is not None else "") + " " + 
						(higher_level_reference_text if higher_level_reference_text is not None else "")
					)
				agency_instance = onto.search_one(iri=f"*{element['agency']}")
				agency_instance.hasNoJurisdiction.append(jurisdiction_instance)
				temp = agency_instance.hasNoJurisdiction

				for i in temp:
					print(i.name)
				if element["agency"] in agency_and_jurisdiction["not_allow"]:
					agency_and_jurisdiction["not_allow"][element["agency"]].append(
						(jurisdiction_text if jurisdiction_text is not None else "") + " " +
						(reference_text if reference_text is not None else "") + " " +
						(higher_level_reference_text if higher_level_reference_text is not None else "")
					)
				else:
					agency_and_jurisdiction["not_allow"][element["agency"]] = [
						jurisdiction_text + " " +
						(reference_text if reference_text else "") + " " +
						(higher_level_reference_text if higher_level_reference_text else "")
					]

with open('67_2014_QH13_259729_only_dieu_jurisdict_agency_jurisdiction.json', 'w', encoding="utf-8") as json_file_4:
	json.dump(agency_and_jurisdiction, json_file_4, ensure_ascii=False, indent=4)

onto.save(file="authority_ontology.owl", format="rdfxml")

 quy định về điều kiện đầu tư kinh doanh.  Điều 7. Ngành, nghề đầu tư kinh doanh có điều kiện
 quy định về điều kiện đầu tư kinh doanh.  Điều 7. Ngành, nghề đầu tư kinh doanh có điều kiện
 quy định về điều kiện đầu tư kinh doanh.  Điều 7. Ngành, nghề đầu tư kinh doanh có điều kiện


* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


In [ ]:
from owlready2 import *

onto = get_ontology("authority_ontology.owl")

onto.load()

print(list(onto.classes()))
test = onto.get_namespace("http://test.org/onto.owl#")
# create a new instance of class Jurisdiction with the name is "giá cước"

agency = onto.search_one(iri="*chính phủ")

temp = agency.hasJurisdiction

for i in temp:
	print(i.name)

In [6]:
from owlready2 import *

# Load your ontology
onto = get_ontology("authority_ontology.owl").load()

# Define the namespace (replace 'onto' with your actual namespace if different)
namespace = onto.get_namespace("http://test.org/onto.owl#")

# SPARQL query to get all instances of class Jurisdiction
query = """
SELECT ?jurisdiction
WHERE {
    ?jurisdiction a <http://test.org/onto.owl#Jurisdiction>
}
"""

# Execute the query
results = list(default_world.sparql(query))

# Print the results
# for result in results:
#     print(result[0].name)
# For each instance of Jurisdiction, print the object properties asserted with it and the individuals related to it
for jurisdiction in namespace.Jurisdiction.instances():
	# print(jurisdiction.name)
	print(jurisdiction.get_relations())
	for property in jurisdiction.get_properties():
		print(property)
		for individual in property[jurisdiction]:
			print(individual.name)

AttributeError: 'get_relations' property is not defined.